# STAT 301 – Final Report (Group)

# 1. Data Description

##  Dataset Source and Collection

The dataset used in this project is titled **“Airbnb Prices in European Cities”**, published on [Kaggle](https://www.kaggle.com/datasets/thedevastator/airbnb-prices-in-european-cities). It was assembled with **scraped data of the publicly available sources of Airbnb listings**. Such listings have specific metadata like the price, the room features, the satisfaction and cleanliness level, the geographic position, and the availability of the host.

Though the original Kaggle dataset lacks any particular code on web scraping, we can infer that the data was scraped by the process of **automated crawling of the Airbnb platform** over a series of cities of Europe on the basis of the structure and the naming of the variables. Every listing represents only one available property at one particular time.



## Dataset Overview

This project focuses on three cities:
- **Amsterdam**
- **Athens**
- **Berlin**

Each city has two files:
- One for **weekday listings**
- One for **weekend listings**

In total, we have **six files**, which were merged into a single dataset using **R**.

In [2]:
# Load libraries
library(readr)
library(dplyr)

# Read in all six CSVs
amsterdam_weekdays <- read_csv("amsterdam_weekdays.csv")
amsterdam_weekends <- read_csv("amsterdam_weekends.csv")
athens_weekdays <- read_csv("athens_weekdays.csv")
athens_weekends <- read_csv("athens_weekends.csv")
berlin_weekdays <- read_csv("berlin_weekdays.csv")
berlin_weekends <- read_csv("berlin_weekends.csv")

New names:
• `` -> `...1`
Rows: 1103 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): room_type
dbl (16): ...1, realSum, person_capacity, multi, biz, cleanliness_rating, gu...
lgl  (3): room_shared, room_private, host_is_superhost

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 977 Columns: 20
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): room_type
dbl (16): ...1, realSum, person_capacity, multi, biz, cleanliness_rating, gu...
lgl  (3): room_shared, room_private, host_is_superhost

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
New names:
• `` -> `...1`
Rows: 2653 Columns: 20
── Column specification ──────────────────

In [3]:
# Add city and day_type to each dataset
amsterdam_weekdays$city <- "Amsterdam"
amsterdam_weekends$city <- "Amsterdam"
athens_weekdays$city <- "Athens"
athens_weekends$city <- "Athens"
berlin_weekdays$city <- "Berlin"
berlin_weekends$city <- "Berlin"

amsterdam_weekdays$day_type <- "weekday"
amsterdam_weekends$day_type <- "weekend"
athens_weekdays$day_type <- "weekday"
athens_weekends$day_type <- "weekend"
berlin_weekdays$day_type <- "weekday"
berlin_weekends$day_type <- "weekend"

# Combine all datasets
airbnb <- bind_rows(
  amsterdam_weekdays,
  amsterdam_weekends,
  athens_weekdays,
  athens_weekends,
  berlin_weekdays,
  berlin_weekends
)

# Convert to factors
airbnb_clean <- airbnb %>%
  mutate(
    city = as.factor(city),
    day_type = as.factor(day_type),
    host_is_superhost = as.factor(host_is_superhost),
    biz = as.factor(biz)
  )
